In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Normalizer
from scipy.special import psi, polygamma
import random
import tensorflow as tf

tf.random.set_seed(0)

In [ ]:
from keras.models import Sequential
from keras.models import Model
from keras import regularizers
from keras.layers import Input, Dense, Dropout, Activation, Lambda
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.layers import LSTM, GRU, SimpleRNN

In [ ]:
#This function is used to encode a vector as follows:
'''
when the vector element == 'normal', it will be transformed to 1 and to 0 if not
'''
def encoding(y):
  enc = [1 if y[i]=='normal' else 0 for i in range(len(y))]
  return pd.DataFrame(enc)

##**Load the Data**

In [ ]:
def load_data(path_train, path_test):
  train = pd.read_csv(path_train)  #load the train data
  test = pd.read_csv(path_test)  #load the test data

  train_normal = pd.DataFrame(train[train['connection_type'] == 'normal'])   #train samples which have normal as label
  train_attack = pd.DataFrame(train[train['connection_type'] != 'normal'])   #train samples which have attack as label
  test_normal = pd.DataFrame(test[test['connection_type'] == 'normal'])      #test samples which have normal as label 
  test_attack = pd.DataFrame(test[test['connection_type'] != 'normal'])      #test samples which have attack as label

  nbr_of_normal_train_samples = 10000  #number of the normal samples that we will work with in the train data
  nbr_of_attack_train_samples = 10000  #number of the attack samples that we will work with in the train data
  nbr_of_normal_test_samples = 2500   #number of the normal samples that we will work with in the test data
  nbr_of_attack_test_samples = 2500    #number of the attack samples that we will work with in the test data
  
  tf.random.set_seed(0)
  train_normal = pd.DataFrame(train_normal[10000:10000+nbr_of_normal_train_samples])
  train_attack = pd.DataFrame(train_attack[10000:10000+nbr_of_attack_train_samples])
  test_normal = pd.DataFrame(test_normal[1500:1500+nbr_of_normal_test_samples])
  test_attack = pd.DataFrame(test_attack[1500:1500+nbr_of_attack_test_samples])

  #concatenate the normal samples and the attack samples into a single data with the desired number of normal samples and attack samples for both train and test
  train = pd.DataFrame(pd.concat([train_normal,train_attack]))   
  test = pd.DataFrame(pd.concat([test_normal,test_attack]))

  train = train.reset_index(drop = True)   #reset index for the train data
  test = test.reset_index(drop = True)     #reset index for the test data

  #split the train and test into features data and labels
  x_train = pd.DataFrame(train.iloc[:,:41])
  y_train = pd.Series(train.iloc[:,41])
  x_test = pd.DataFrame(test.iloc[:,:41])
  y_test = pd.Series(test.iloc[:,41])

  #use the command 'get_dummies' to eliminate the categorical features
  x_train = pd.get_dummies(x_train)
  x_test = pd.get_dummies(x_test)

  #join x_train and x_test as both of them they haven't the same shape after 'get_dummies' command
  x_train , x_test = x_train.align(x_test, join = 'inner', axis = 1) 

  #encode both y_train and y_test 
  y_train = encoding(y_train)
  y_test = encoding(y_test)

  #Normalize the data
  scaler = Normalizer().fit(x_train)
  x_train = scaler.transform(x_train)

  scaler = Normalizer().fit(x_test)
  x_test = scaler.transform(x_test)

  #transform y_train and y_test into arrays
  y_train = np.array(y_train)
  y_test = np.array(y_test)

  return (x_train , y_train) , (x_test , y_test)

##**Transformtions**

In [ ]:
def transformation(input_dim, h_dim,):
  input = Input(shape=(input_dim,))
  layer1 = Dense(h_dim, use_bias = False, activation='relu')(input)
  output = Dense(input_dim, use_bias = False, activation='relu')(layer1)

  # this model maps an input to its reconstruction
  transformer = Model(input, output)
  return transformer

In [ ]:
def create_transformations_list(input_dim, h_dim, num_trans):
  trans_list = []
  for i in range(num_trans):
    trans = transformation(input_dim ,h_dim[i])
    trans_list.append(trans)
  return trans_list 

##**Encoding**

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

def AutoEncoder(input_shape):
  input = Input(shape=(input_shape,))
  layer1 = Dense(128, use_bias = False, activation='relu')(input)
  layer2 = Dense(64, use_bias = False, activation='relu')(layer1)
  encoded = Dense(32, use_bias = False, activation='relu')(layer2)  # encoding_dim = 32
  layer3 = Dense(64, use_bias = False, activation='relu')(encoded)
  layer4 = Dense(128, use_bias = False, activation='relu')(layer3)
  decoded = Dense(input_shape, use_bias = False, activation='relu')(layer4)

  # this model maps an input to its reconstruction
  autoencoder = Model(input, decoded)
  return autoencoder

In [ ]:
def classifier(input_shape,lstm_output_size,nbr_of_classes):
  model = Sequential()
  model.add(Convolution1D(128, 3, padding="same",activation="relu",input_shape=(input_shape, 1)))
  model.add(MaxPooling1D(pool_size=3))
  model.add(LSTM(lstm_output_size))
  model.add(Dropout(0.1))
  model.add(Dense(nbr_of_classes, activation="softmax"))
  return model

##**Experiments**

In [ ]:
def calc_approx_alpha_sum(observations):
        N = observations.shape[0]
        K = observations.shape[1]
        f = np.mean(observations, axis=0)
        log_obs = (np.nan_to_num(np.log(np.clip(observations,10**(-10),None)), copy=True, nan=10, posinf=10**10, neginf=-(10**10)))

        return (N * (K - 1) * (-psi(1))) / (
                N * sum(f * np.log(f)) + sum([np.log(sum(observations[j][:K-1])) for j in range(N)]) - sum([sum(f*log_obs[j]) for j in range(N)]) )

def inv_psi(y, iters=5):
        # initial estimate
        cond = y >= -2.22
        x = cond * (np.exp(y) + 0.5) + (1 - cond) * -1 / (y - psi(1))

        for _ in range(iters):
            x = x - (psi(x) - y) / polygamma(1, x)
        return x

def fixed_point_beta_louiville_mle(vect_alpha_0, alpha_0, beta_0,y, max_iter=100):
        log_y = np.nan_to_num(np.log(np.clip(y,10**(-10),None)), copy=True, nan=1, posinf=10**10, neginf=-(10**10)).mean(axis=0)
        N = y.shape[0]
        K = y.shape[1]
        alpha_old = alpha_0
        beta_old = beta_0
        vect_alpha_old = vect_alpha_0

        for iterr in range(max_iter):
          #calculate alpha
          aph = psi(alpha_old + beta_old) + (1/N)*sum([np.log(sum(y[j,:K-1])) for j in range(N)]) 
          aph = np.nan_to_num(aph, copy=True, nan=1, posinf=10**10, neginf=-(10**10))
          alpha_new = inv_psi(np.clip(aph,None,7), iters=5)
          alpha_new = np.clip(alpha_new,10**(-10),None)

          #calculate beta
          bet = psi(alpha_old + beta_old) + log_y[K-1]
          bet = np.nan_to_num(bet, copy=True, nan=1, posinf=10**10, neginf=-(10**10))
          beta_new = inv_psi(np.clip(bet,None,7), iters=5)
          beta_new = np.clip(beta_new,10**(-10),None)

          #calculate the vector alpha
          vect_alpha_new = np.asarray([inv_psi(psi(sum(vect_alpha_old)) + log_y[i] - (1/N)*sum([np.log(sum(y[j,:K-1])) for j in range(N)]) ,iters=5) for i in range(K)])
          vect_alpha_new = np.nan_to_num((np.clip(vect_alpha_new,10**(-10),None)), copy=True, nan=1, posinf=10**10, neginf=-(10**10))
          if np.sqrt(np.sum((vect_alpha_old - vect_alpha_new) ** 2)) < 1e-7 and np.sqrt((alpha_new - alpha_old) ** 2) < 1e-7 and np.sqrt((beta_new - beta_old) ** 2) < 1e-7: 
            break
          #update the parameters
          alpha_old = alpha_new
          beta_old = beta_new
          vect_alpha_old = vect_alpha_new
        return vect_alpha_new, alpha_new, beta_new

def beta_louiville_normality_score(vect_alpha,alpha,beta,y):
      N=y.shape[0]
      K=y.shape[1]
      log_y = np.nan_to_num(np.log(np.clip(y,10**(-10),None)), copy=True, nan=0.5, posinf=10**10, neginf=-(10**10))
      scores = np.asarray([sum((vect_alpha-1)*log_y[j]) + (beta-1)*log_y[j][K-1] + (alpha-sum(vect_alpha))*np.log(sum(y[j][:K-1]))  for j in range(N)]) #+ np.log(gamma(sum(vect_alpha))) + np.log(gamma(alpha+beta)) - np.log(gamma(alpha)) - np.log(gamma(beta)) - sum([np.log(gamma(vect_alpha[i])) for i in range(K)])
      scores = np.nan_to_num(scores, copy=True, nan=10**5, posinf=10**10, neginf=-(10**10))
      return scores


This cell for calculating our metrics

In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc
def roc_pr_curve_data(scores, labels, single_class_ind):
    scores = scores.flatten()
    labels = labels.flatten()

    scores_pos = scores[labels == 1]
    scores_neg = scores[labels != 1]

    truth = np.concatenate((np.zeros_like(scores_neg), np.ones_like(scores_pos)))
    preds = np.concatenate((scores_neg, scores_pos))
    fpr, tpr, roc_thresholds = roc_curve(truth, preds)
    roc_auc = auc(fpr, tpr)
    print("roc_auc for class:",single_class_ind,'=',roc_auc,'\n')

    # pr curve where "normal" is the positive class
    precision_norm, recall_norm, pr_thresholds_norm = precision_recall_curve(truth, preds)
    pr_auc_norm = auc(recall_norm, precision_norm)
    print("pr_auc_norm for class",single_class_ind,"where normal is the positive class",pr_auc_norm,'\n')

    # pr curve where "anomaly" is the positive class
    precision_anom, recall_anom, pr_thresholds_anom = precision_recall_curve(truth, -preds, pos_label=0)
    pr_auc_anom = auc(recall_anom, precision_anom)
    print("pr_auc_anom for class",single_class_ind,"where anomaly is the positive class",pr_auc_anom,'\n')

In [ ]:
#load the data
path_train = "train_kdd.csv"
path_test = "test_kdd.csv"
(x_train, y_train) , (x_test, y_test) = load_data(path_train, path_test)

In [ ]:
import os
import csv
from collections import defaultdict
from glob import glob
from multiprocessing import Manager, freeze_support, Process

In [ ]:
def experiment(single_class_ind,gpu_q):
  #print("Class ",single_class_ind)
  gpu_to_use = gpu_q.get()
  os.environ["CUDA_VISIBLE_DEVICES"] = gpu_to_use
  tf.random.set_seed(0)


  #Transform the data
  print('Start of transformations for class',single_class_ind,'\n')
  x_train_task = x_train[y_train.flatten() == single_class_ind]
  num_trans = 10
  h_dim = [10,20,30,40,50,60,70,80,90,100]
  transformations_inds = np.tile(np.arange(num_trans), len(x_train_task))
  transformations_list = create_transformations_list(x_train_task.shape[1], h_dim, num_trans)
  data_each_transformation = []
  for transformation in transformations_list:
    print('transformation ', transformations_list.index(transformation))
    transformation.compile(loss="mse", optimizer="adamax",metrics=['accuracy'])
    transformation.fit(x_train_task, x_train_task, batch_size=64, epochs= 10) 
    data_trans = transformation.predict(x_train_task)
    data_each_transformation.append(data_trans)
    print('\n')
  data_transformed = pd.concat([pd.DataFrame(data_each_transformation[i]) for i in range(len(data_each_transformation))])
  data_transformed = data_transformed.reset_index(drop = True)
  data_organised = []
  for i in range(len(x_train_task)):
    data_trans_k = data_transformed.iloc[i::len(x_train_task),:]
    data_organised.append(data_trans_k)
  x_train_task_transformed = pd.concat([pd.DataFrame(data_organised[i]) for i in range(len(data_organised))])
  x_train_task_transformed = x_train_task_transformed.reset_index(drop = True)
  print('End of transformations for class',single_class_ind,'\n')


  #Encode the data
  print('Start of AutoEncoding for class',single_class_ind,'\n')
  autoencoder = AutoEncoder(x_train_task_transformed.shape[1])
  autoencoder.compile(optimizer='adam', loss='mse')
  autoencoder.fit(x_train_task_transformed, x_train_task_transformed, epochs=3, batch_size=64)
  x_train_task_encoded = autoencoder.predict(x_train_task_transformed)
  print('End of AutoEncoding for class',single_class_ind,'\n')

  #Create and train the classifier
  cls = classifier(x_train_task_transformed.shape[1],70,num_trans)
  cls.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])
  cls.fit(x=x_train_task_encoded, y=to_categorical(transformations_inds),
            batch_size=64, epochs= 250)    #N_EPOCHS

  scores = np.zeros((len(x_test),))
  observed_data = x_train_task
  for t_ind in range(num_trans):
    observed_data_transformed = data_each_transformation[t_ind]
    observed_data_autoencoded = autoencoder.predict(observed_data_transformed)
    observed_dirichlet = cls.predict(observed_data_autoencoded) 

    alpha_sum_approx = np.abs(calc_approx_alpha_sum(observed_dirichlet))
    alpha_sum_approx = np.nan_to_num(alpha_sum_approx, copy=True, nan=1, posinf=10**10, neginf=-10*10)
    vect_alpha_0 = np.clip(observed_dirichlet.mean(axis=0) * alpha_sum_approx,10**(-10),10**10)
    alpha_0 = 0.5
    beta_0 = 0.001

    mle_vect_alpha_t,mle_alpha_t,mle_beta_t = fixed_point_beta_louiville_mle(vect_alpha_0, alpha_0, beta_0,observed_dirichlet,max_iter=100)
    mle_vect_alpha_t=np.nan_to_num(mle_vect_alpha_t, copy=True, nan=1, posinf=10**10, neginf=-10*10)
    mle_alpha_t= np.nan_to_num(mle_alpha_t, copy=True, nan=10**3, posinf=10**10, neginf=-10*10)
    mle_beta_t= np.nan_to_num(mle_beta_t, copy=True, nan=10**3, posinf=10**10, neginf=-10**10)

    x_test_p_transformed = transformations_list[t_ind].predict(x_test)
    x_test_p_autoencoded = autoencoder.predict(x_test_p_transformed)
    x_test_p = cls.predict(x_test_p_autoencoded)
    x_test_p = np.nan_to_num(x_test_p, copy=True, nan=0, posinf=None, neginf=None)
    scores += beta_louiville_normality_score(mle_vect_alpha_t,mle_alpha_t,mle_beta_t, x_test_p)
  
  scores /= num_trans
  labels = y_test.flatten() == single_class_ind
  roc_pr_curve_data(scores, labels, single_class_ind)
  gpu_q.put(gpu_to_use)

In [ ]:
def run_experiments(dataset_name,q, n_classes):
    n_runs = 2
    # Transformations
    for i in range(n_runs):
        print('Run number:',i+1,'\n')
        processes = [Process(target=experiment,
                             args=(c, q)) for c in range(n_classes)]

        for p in processes:
            p.start()
        for p in processes:
            p.join()

In [ ]:
if __name__ == '__main__':
    freeze_support()
    N_GPUS = 1
    man = Manager()
    q = man.Queue(N_GPUS)
    for g in range(N_GPUS):
        q.put(str(g))

    experiments_list = [ 
        ('kdd_cup', 2),
    ]

    for dataset_name, n_classes in experiments_list:
        run_experiments(dataset_name, q, n_classes)

Run number: 1 

Start of transformations for class 0 

transformation  0
Epoch 1/10
157/157 [==============================] - 3s 2ms/step - loss: 0.0088 - accuracy: 0.1010
Epoch 2/10
157/157 [==============================] - 0s 2ms/step - loss: 0.0074 - accuracy: 0.7072
Epoch 3/10
157/157 [==============================] - 0s 2ms/step - loss: 0.0050 - accuracy: 0.7509
Epoch 4/10
157/157 [==============================] - 0s 2ms/step - loss: 0.0030 - accuracy: 0.7539
Epoch 5/10
157/157 [==============================] - 0s 2ms/step - loss: 0.0013 - accuracy: 0.7809
Epoch 6/10
157/157 [==============================] - 0s 2ms/step - loss: 7.9452e-04 - accuracy: 0.7951
Epoch 7/10
157/157 [==============================] - 0s 2ms/step - loss: 5.2774e-04 - accuracy: 0.8158
Epoch 8/10
157/157 [==============================] - 0s 2ms/step - loss: 3.2373e-04 - accuracy: 0.8349
Epoch 9/10
157/157 [==============================] - 0s 2ms/step - loss: 1.9081e-04 - accuracy: 0.9204
Epoch 10/10